## 1. 卷积层尺寸计算原理

输入矩阵格式：四个维度 (N,C,H,W)，依次为：样本数、通道数、图像高度、图像宽度  

输出矩阵格式：与输出矩阵的维度顺序和含义相同，但是后三个维度（图像通道数、图像高度、图像宽度）的尺寸发生变化  

权重矩阵（卷积核）格式：同样是四个维度 (filter_height, filter_width, in_channels, out_channels)，但维度的含义与上面不同，为：卷积核高度、卷积核宽度、输入通道数、输出通道数  

输入矩阵、权重矩阵、输出矩阵这三者之间的相互决定关系
- 卷积核的输入通道数（in depth）由输入矩阵的通道数所决定。
- 输出矩阵的通道数（out depth）由卷积核的输出通道数所决定。
- 输出矩阵的高度和宽度（height, width）这两个维度的尺寸由输入矩阵、卷积核、扫描方式共同决定。  

计算公式如下 $\begin{cases} H_{out} &= (H_{in} - H_{kernel} + 2 * padding) ~ / ~ stride + 1\\[2ex] W_{out} &= (W_{in} - W_{kernel} + 2 * padding) ~ / ~ stride + 1 \end{cases} $

## 2. 卷积实现  

用矩阵乘法实现：卷积运算本质上就是在滤波器和输入数据的局部区域间做点积。卷积层的常用实现方式就是利用这一点，将卷积层的前向传播变成一个巨大的矩阵乘法：  

- 输入图像的局部区域被 im2col 操作拉伸为列。比如，如果输入是 [227x227x3]，要与尺寸为 11x11x3 的滤波器以步长为 4 进行卷积，就取输入中的 [11x11x3] 数据块，然后将其拉伸为长度为 11x11x3=363 的列向量。重复进行这一过程，因为步长为 4，所以输出的宽高为 (227-11)/4+1=55，所以得到im2col 操作的输出矩阵 X_col 的尺寸是 [363x3025]，其中每列是拉伸的感受野，共有 55x55=3,025 个。注意因为感受野之间有重叠，所以输入数据体中的数字在不同的列中可能有重复。  
- 卷积层的权重也同样被拉伸成行。举例，如果有 96 个尺寸为 [11x11x3] 的滤波器，就生成一个矩阵 W_row，尺寸为 [96x363]。  
- 现在卷积的结果和进行一个大矩阵乘 np.dot(W_row, X_col) 是等价的了，能得到每个滤波器和每个感受野间的点积。在我们的例子中，这个操作的输出是 [96x3025]，给出了每个滤波器在每个位置的点积输出。  
- 结果最后必须被重新变为合理的输出尺寸 [55x55x96]。

这个方法的缺点就是占用内存太多，因为在输入数据体中的某些值在 X_col 中被复制了多次。但是，其优点是矩阵乘法有非常多的高效实现方式，我们都可以使用（比如常用的BLAS API）。还有，同样的 im2col 思路可以用在 pooling 操作中。

反向传播：卷积操作的反向传播（同时对于数据和权重）还是一个卷积（但是是和空间上翻转的滤波器）。

In [1]:
def conv_forward_naive(x, w, b, conv_param):
  """
  A naive implementation of the forward pass for a convolutional layer.

  The input consists of N data points, each with C channels, height H and width
  W. We convolve each input with F different filters, where each filter spans
  all C channels and has height HH and width HH.

  Input:
  - x: Input data of shape (N, C, H, W)
  - w: Filter weights of shape (F, C, HH, WW)
  - b: Biases, of shape (F,)
  - conv_param: A dictionary with the following keys:
    - 'stride': The number of pixels between adjacent receptive fields in the
      horizontal and vertical directions.
    - 'pad': The number of pixels that will be used to zero-pad the input.

  Returns a tuple of:
  - out: Output data, of shape (N, F, H', W') where H' and W' are given by
    H' = 1 + (H + 2 * pad - HH) / stride
    W' = 1 + (W + 2 * pad - WW) / stride
  - cache: (x, w, b, conv_param)
  """
  out = None
  N, C, H, W = x.shape
  F, _, HH, WW = w.shape
  stride, pad = conv_param['stride'], conv_param['pad']
  H_out = 1 + (H + 2 * pad - HH) / stride
  W_out = 1 + (W + 2 * pad - WW) / stride
  out = np.zeros((N , F , H_out, W_out))

  x_pad = np.pad(x, ((0,), (0,), (pad,), (pad,)), mode='constant', constant_values=0)
  for i in range(H_out):
      for j in range(W_out):
          x_pad_masked = x_pad[:, :, i*stride:i*stride+HH, j*stride:j*stride+WW]
          for k in range(F):
              out[:, k , i, j] = np.sum(x_pad_masked * w[k, :, :, :], axis=(1,2,3)) 
  out = out + (b)[None, :, None, None]
  cache = (x, w, b, conv_param)
  return out, cache


def conv_backward_naive(dout, cache):
  """
  A naive implementation of the backward pass for a convolutional layer.

  Inputs:
  - dout: Upstream derivatives.
  - cache: A tuple of (x, w, b, conv_param) as in conv_forward_naive

  Returns a tuple of:
  - dx: Gradient with respect to x
  - dw: Gradient with respect to w
  - db: Gradient with respect to b
  """
  dx, dw, db = None, None, None
  x, w, b, conv_param = cache
  
  N, C, H, W = x.shape
  F, _, HH, WW = w.shape
  stride, pad = conv_param['stride'], conv_param['pad']
  H_out = 1 + (H + 2 * pad - HH) / stride
  W_out = 1 + (W + 2 * pad - WW) / stride
  
  x_pad = np.pad(x, ((0,), (0,), (pad,), (pad,)), mode='constant', constant_values=0)
  dx = np.zeros_like(x)
  dx_pad = np.zeros_like(x_pad)
  dw = np.zeros_like(w)
  db = np.zeros_like(b)
  
  db = np.sum(dout, axis = (0,2,3))
  
  x_pad = np.pad(x, ((0,), (0,), (pad,), (pad,)), mode='constant', constant_values=0)
  for i in range(H_out):
      for j in range(W_out):
          x_pad_masked = x_pad[:, :, i*stride:i*stride+HH, j*stride:j*stride+WW]
          for k in range(F): #compute dw
              dw[k ,: ,: ,:] += np.sum(x_pad_masked * (dout[:, k, i, j])[:, None, None, None], axis=0)
          for n in range(N): #compute dx_pad
              dx_pad[n, :, i*stride:i*stride+HH, j*stride:j*stride+WW] += np.sum((w[:, :, :, :] * 
                                                 (dout[n, :, i, j])[:,None ,None, None]), axis=0)
  dx = dx_pad[:,:,pad:-pad,pad:-pad]
  return dx, dw, db


def conv_forward_im2col(x, w, b, conv_param):
  """
  A fast implementation of the forward pass for a convolutional layer
  based on im2col and col2im.
  """
  N, C, H, W = x.shape
  num_filters, _, filter_height, filter_width = w.shape
  stride, pad = conv_param['stride'], conv_param['pad']

  # Check dimensions
  assert (W + 2 * pad - filter_width) % stride == 0, 'width does not work'
  assert (H + 2 * pad - filter_height) % stride == 0, 'height does not work'

  # Create output
  out_height = (H + 2 * pad - filter_height) / stride + 1
  out_width = (W + 2 * pad - filter_width) / stride + 1
  out = np.zeros((N, num_filters, out_height, out_width), dtype=x.dtype)

  # x_cols = im2col_indices(x, w.shape[2], w.shape[3], pad, stride)
  x_cols = im2col_cython(x, w.shape[2], w.shape[3], pad, stride)
  res = w.reshape((w.shape[0], -1)).dot(x_cols) + b.reshape(-1, 1)

  out = res.reshape(w.shape[0], out.shape[2], out.shape[3], x.shape[0])
  out = out.transpose(3, 0, 1, 2)

  cache = (x, w, b, conv_param, x_cols)
  return out, cache

def conv_backward_im2col(dout, cache):
  """
  A fast implementation of the backward pass for a convolutional layer
  based on im2col and col2im.
  """
  x, w, b, conv_param, x_cols = cache
  stride, pad = conv_param['stride'], conv_param['pad']

  db = np.sum(dout, axis=(0, 2, 3))

  num_filters, _, filter_height, filter_width = w.shape
  dout_reshaped = dout.transpose(1, 2, 3, 0).reshape(num_filters, -1)
  dw = dout_reshaped.dot(x_cols.T).reshape(w.shape)

  dx_cols = w.reshape(num_filters, -1).T.dot(dout_reshaped)
  # dx = col2im_indices(dx_cols, x.shape, filter_height, filter_width, pad, stride)
  dx = col2im_cython(dx_cols, x.shape[0], x.shape[1], x.shape[2], x.shape[3],
                     filter_height, filter_width, pad, stride)

  return dx, dw, db

## 3. 卷积类型  

### 3.1 1x1卷积  

1x1 卷积和正常的卷积一样，唯一不同的是它的大小是 1x1，没有考虑 feature map 局部信息之间的关系。最早出现在 Network In Network 的论文中 ，使用 1x1 卷积是想加深加宽网络结构 ，在 Inception 网络（ Going Deeper with Convolutions ）中用来降维。  

1x1 卷积的主要作用有以下几点：
- 1、降维: 比如，一张 500 * 500 且 depth 为 100 的图片在 20 个 filter 上做 1x1 的卷积，那么结果的大小为500x500x20。
- 2、加入非线性: 卷积层之后经过激励层，1x1 的卷积在前一层的学习表示上添加了非线性激励，提升网络的表达能力


### 3.2 Dilation 卷积  

Dilation卷积，通常译作空洞卷积或者卷积核膨胀操作，它是解决 pixel-wise 输出模型的一种常用的卷积方式。一种普遍的认识是，pooling 下采样操作导致的信息丢失是不可逆的，通常的分类识别模型，只需要预测每一类的概率，所以我们不需要考虑 pooling 会导致损失图像细节信息的问题，但是做像素级的预测时（譬如语义分割），就要考虑到这个问题了。  

dilated 的好处是不做 pooling 损失信息的情况下，加大了感受野，让每个卷积输出都包含较大范围的信息通过卷积核插“0”的方式，它可以比普通的卷积获得更大的感受野。在图像需要全局信息或者语音文本需要较长的 sequence 信息依赖的问题中，都能很好的应用 dilated conv，比如图像分割、语音合成WaveNet、机器翻译ByteNet中。

一个扩张率为 2 的 3×3 卷积核，感受野与 5×5 的卷积核相同，而且仅需要 9 个参数。可以把它想象成一个 5×5 的卷积核，每隔一行或一列删除一行或一列。在相同的计算条件下，空洞卷积提供了更大的感受野。当网络层需要较大的感受野，但计算资源有限而无法提高卷积核数量或大小时，可以考虑空洞卷积。


### 3.3 深度可分离卷积  

深度卷积是对输入的每一个 channel 独立的用所有卷积核去卷积，假设卷积核的 shape 是 [filter_height, filter_width, in_channels, channel_multiplier]，那么每个 in_channel 会输出 channel_multiplier 那么多个通道，最后的 feature map 就会有 in_channels * channel_multiplier 个通道了。反观普通的卷积，输出的 feature map 一般就只有 channel_multiplier 那么多个通道。

既然叫深度可分离卷积，光做 depthwise convolution 肯定是不够的，原文在深度卷积后面又加了 pointwise convolution，这个 pointwise convolution 就是 1x1 的卷积，可以看做是对那么多分离的通道做了个融合。

这两个过程合起来，就称为 Depthwise Separable Convolution了，这种方法在保持通道分离的前提下，接上一个深度卷积结构，即可实现空间卷积。接下来通过一个例子可以更好地理解。

假设有一个 3×3 大小的卷积层，其输入通道为 16、输出通道为 32。具体为，32 个 3×3 大小的卷积核会遍历 16 个通道中的每个数据，从而产生16×32=512 个特征图谱。进而通过叠加每个输入通道对应的特征图谱后融合得到 1 个特征图谱。最后可得到所需的 32 个输出通道。

针对这个例子应用深度可分离卷积，用 1 个 3×3 大小的卷积核遍历 16 通道的数据，得到了 16 个特征图谱。在融合操作之前，接着用 32 个 1×1 大小的卷积核遍历这 16 个特征图谱，进行相加融合。这个过程使用了 16×3×3+16×32×1×1=656 个参数，远少于上面的 16×32×3×3=4608 个参数。

这个例子就是深度可分离卷积的具体操作，从 Xception 模型的效果可以看出，这种方法是比较有效的。由于能够有效利用参数，因此深度可分离卷积也可以用于移动设备中。

### 3.4 可变形卷积  

可形变卷积认为规则形状的卷积核（比如 3x3 卷积）可能会限制特征的提取，如果赋予卷积核形变的特性，让网络根据 label 反传下来的误差自动的调整卷积核的形状，适应网络重点关注的感兴趣的区域，就可以提取更好的特征。

如下图：网络会根据原位置（a），学习一个 offset 偏移量，得到新的卷积核（b）（c）（d），那么一些特殊情况就会成为这个更泛化的模型的特例，例如图（c）表示从不同尺度物体的识别，图（d）表示旋转物体的识别。

![Aaron Swartz](https://github.com/liyibo/cv_notebooks/blob/master/markdown_pics/train_model/convs/Deformable.jpg?raw=true)  

可形变卷积将原来的一次卷积变为两次卷积，第一次是获取 offsets 的卷积，即对 input feature map 做卷积，得到一个输出（offset field），然后再在这个输出上取对应位置的一组值作为 offsets。假设 input feature map 的 shape 为 [batch，height，width，channels]，指定输出通道变成两倍，卷积得到的 offset field 就是 [batch，height，width，2×channels]，为什么指定通道变成两倍呢？因为需要在这个 offset field 里面取一组卷积核的 offsets，而一个 offset 肯定不能一个值就表示的，最少也要用两个值（x方向上的偏移和y方向上的偏移）所以，如果我们的卷积核是3x3，那意味着需要 3x3 个 offsets，一共需要 2x3x3 个值，取完了这些值，就可以顺利使卷积核形变了。第二处就是使用变形的卷积核来卷积，这个比较常规。（这里还有一个用双线性插值的方法获取某一卷积形变后位置的输入的过程）

## 参考

1. [CNN中卷积层的计算细节](https://zhuanlan.zhihu.com/p/29119239)  
2. [CS231n官方笔记授权翻译总集](https://zhuanlan.zhihu.com/p/21930884?refer=intelligentunit)  
3. [CNN中千奇百怪的卷积方式大汇总](https://zhuanlan.zhihu.com/p/29367273)  
4. [一文了解各种卷积结构原理及优劣](https://baijiahao.baidu.com/s?id=1574230930627497&wfr=spider&for=pc)  
5. [卷积神经网络中用1*1 卷积有什么作用或者好处呢？](https://www.zhihu.com/question/56024942)  
6. [变形卷积核、可分离卷积？卷积神经网络中十大拍案叫绝的操作](https://zhuanlan.zhihu.com/p/28749411)